# BIXI data analysis 2024

In [9]:
import pandas as pd
import geopandas as gpd
import leafmap

In [3]:
bixi = pd.read_csv('../data/raw/bixi/bixi-2024.csv')

In [4]:
bixi.head()

,STARTSTATIONNAME,STARTSTATIONARRONDISSEMENT,STARTSTATIONLATITUDE,STARTSTATIONLONGITUDE,ENDSTATIONNAME,ENDSTATIONARRONDISSEMENT,ENDSTATIONLATITUDE,ENDSTATIONLONGITUDE,STARTTIMEMS,ENDTIMEMS
0,Métro Champ-de-Mars (Viger / Sanguinet),Ville-Marie,45.510253,-73.556777,St-Antoine / Jean-Paul Riopelle,Ville-Marie,45.502940,-73.560640,1704230756167,1.704231e+12
1,Métro Place-d'Armes (Viger / St-Urbain),Ville-Marie,45.506314,-73.559671,de Maisonneuve / Aylmer (est),Ville-Marie,45.505386,-73.570432,1704232888826,1.704233e+12
2,Émile-Duployé / Sherbrooke,Le Plateau-Mont-Royal,45.527195,-73.564526,Parc Molson (d'Iberville / Beaubien),Rosemont - La Petite-Patrie,45.549022,-73.591982,1704238154678,1.704239e+12
3,Marmier / St-Denis,Rosemont - La Petite-Patrie,45.531027,-73.598623,Roy / St-André,Le Plateau-Mont-Royal,45.521564,-73.570367,1704232695740,1.704233e+12
4,Marmier / St-Denis,Rosemont - La Petite-Patrie,45.531027,-73.598623,Parc Molson (d'Iberville / Beaubien),Rosemont - La Petite-Patrie,45.549022,-73.591982,1704243796167,1.704244e+12


In [1]:
bixi.shape[0]

NameError: name 'bixi' is not defined

In [14]:
# Create a bixi station dataframe
bixi_stations = bixi[['STARTSTATIONNAME', 'STARTSTATIONLATITUDE', 'STARTSTATIONLONGITUDE']].drop_duplicates()
bixi_stations.head()

,STARTSTATIONNAME,STARTSTATIONLATITUDE,STARTSTATIONLONGITUDE
0,Métro Champ-de-Mars (Viger / Sanguinet),45.510253,-73.556777
1,Métro Place-d'Armes (Viger / St-Urbain),45.506314,-73.559671
2,Émile-Duployé / Sherbrooke,45.527195,-73.564526
3,Marmier / St-Denis,45.531027,-73.598623
5,du Parc-Lafontaine / Rachel,45.525512,-73.574245


In [15]:
# Create a GeoDataFrame from the bixi stations
gdf_bixi_stations = gpd.GeoDataFrame(bixi_stations, geometry=gpd.points_from_xy(bixi_stations.STARTSTATIONLONGITUDE, bixi_stations.STARTSTATIONLATITUDE))

In [20]:
gdf_bixi_stations = gpd.GeoDataFrame(
    bixi_stations, 
    geometry=gpd.points_from_xy(bixi_stations['STARTSTATIONLONGITUDE'], bixi_stations['STARTSTATIONLATITUDE']),
    crs='EPSG:4326'
)
gdf_bixi_stations = gdf_bixi_stations.to_crs(epsg=3857)
gdf_bixi_stations['geometry'] = gdf_bixi_stations['geometry'].buffer(50)  # Adjust the buffer size as needed

In [23]:
m = leafmap.Map(center=[45.5, -73.6], zoom=11)
m.add_basemap("Google Maps")
m

Map(center=[45.5, -73.6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [24]:
m.add_gdf(gdf_bixi_stations, layer_name="Bixi Stations", fill_colors=["Red"], radius=10)

In [25]:
# Export geo json
gdf_bixi_stations.to_file('../data/curated/bixi_stations.geojson', driver='GeoJSON')